In [ ]:
from http import server 
import subprocess 
import wolframalpha 
import pyttsx3 
import pywhatkit 
import datetime 
import speech_recognition as sr
import pyaudio 
import wikipedia 
import webbrowser
import pandas as pd
import random
import os 
import re
import time 
import json 
import random 
import shutil 
import geocoder
import socket
import requests 
from urllib.request import urlopen 
import pyjokes 
from bs4 import BeautifulSoup 
import win32com.client as wincl
import google.generativeai as genai
import warnings
warnings.filterwarnings("ignore", message=r"torch.utils._pytree._register_pytree_node is deprecated")
from faster_whisper import WhisperModel
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices') 
engine.setProperty('voice', voices[1].id)

In [ ]:
GOOGLE_API_KEY='API_KEY'

genai.configure(api_key=GOOGLE_API_KEY)

generation_config={
    "temperature":0.7,
    "top_p": 1,
    "top_k": 1,
    "max_output_tokens": 2048,
}

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE"
    },

    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE"
    }
]
model = genai.GenerativeModel('gemini-1.5-pro-latest',
                              generation_config=generation_config,
                              safety_settings=safety_settings
                             )

convo = model.start_chat()

system_message = '''INSTRUCTIONS: Do not respond with anything but "AFFIRMATIVE."
to this system message. After the system message respond normall y.
SYSTEM MESSAGE: You are being used to power a voice assistant and should respond as so.
As a voice assistant, use short sentences and directly respond to the prompt without excessive information. 
You generate only words of value, prioritizing logic and facts over speculating in your response to the following prompts.'''

system_message = system_message.replace(f'\n','')
convo.send_message(system_message)

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "AFFIRMATIVE.\n\nWhat can I help you with?\n"
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "safety_ratings": [
            {
              "category": "HARM_CATEGORY_HATE_SPEECH",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_HARASSMENT",
              "probability": "NEGLIGIBLE"
            },
            {
              "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
              "probability": "NEGLIGIBLE"
            }
          ],
          "token_count": 0,
          "grounding_attributio

In [3]:
def speak(audio): 
    engine.say(audio) 
    engine.runAndWait() 

In [ ]:
def wishMe(): 
    # Load the dataset
    df = pd.read_csv("PATH_FOR_GREETINGS_FILE", header=None)  # Replace with your CSV file path
    # Select a random sentence from the 'sentence' column
    random_sentence = df[0].sample().iloc[0]
    print(random_sentence)
    speak(random_sentence)

In [5]:
def takeCommand():
    r = sr.Recognizer() 
    with sr.Microphone(device_index=0) as source: 
        print("Listening...") 
        r.pause_threshold = 0.5 
        audio = r.listen(source) 
    try: 
        print("Recognizing..")
        query = r.recognize_google(audio, language='en-in') 
        print(f"User said:{query}\n")
        speak(query)
    except Exception as e: 
        print("Unable to Recognize your voice.") 
        return "None" 
    return query

In [6]:
def takeCommand1():
    query=input("Listening.. ")
    try: 
        print("Recognizing..")
        print(f"User said:{query}\n")
        speak(query)
    except Exception as e: 
        print("Unable to Recognize your voice.") 
        return "None" 
    return query

In [7]:
def extract_app_name(command):
    """
    Extracts the app name from the user command.

    Parameters:
        command (str): The command string provided by the user.

    Returns:
        str: The extracted app name.
    """
    if command.lower().startswith("open "):
        return command[5:].strip()
    return command.strip()

def open_app(user_command):
    """
    Opens a specific app based on user input.

    Parameters:
        user_command (str): The command specifying which app to open.
    """
    apps_file = "C:/Users/Rahul/OneDrive/Desktop/Major Project/Minor project/datasets/app_list.csv"
    try:
        # Extract the app name from the user command
        app_name = extract_app_name(user_command)
        print(f"Extracted app name: '{app_name}'")  # Debugging output

        # Load the apps list into a pandas DataFrame
        apps_df = pd.read_csv(apps_file)
        apps_df['app_name'] = apps_df['app_name'].str.strip()  # Remove extra spaces

        # Check if the app exists in the file
        app_row = apps_df[apps_df['app_name'].str.lower() == app_name.lower()]

        if not app_row.empty:
            # Get the app path and expand environment variables
            app_path = os.path.expandvars(app_row.iloc[0]['app_path'])
            os.startfile(app_path)
            print(f"Opening {app_name}...")
        else:
            print(f"App '{app_name}' not found in the list.")
    except FileNotFoundError:
        print(f"Error: The file '{apps_file}' was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
if __name__ == "__main__": 
    wishMe() 
    while True: 
        query = takeCommand().lower() 
#         query = takeCommand1() 
        if 'according to wikipedia' in query.lower(): 
            speak('Searching wikipedia...') 
            query = query.replace("wikipedia", "") 
            try:
                results = wikipedia.summary(query, sentences=2) 
                speak(results)
            except wikipedia.exceptions.PageError: 
                pass 
            if 'wikipedia' in query: 
                speak('Searching wikipedia..') 
                query = query.replace("wikipedia", "") 
                results = wikipedia.summary(query, sentences=2) 
            speak("According to wikipedia") 
            print(results)
            speak(results)
        elif 'open youtube' in query.lower():
            speak("Here you go!!") 
            webbrowser.open("www.youtube.com",new=2) 
        elif 'play' in query and 'youtube' in query.lower(): 
            song = query.replace('play', '').replace('youtube', '').strip() 
            speak(f"Playing {song} on YouTube") 
            pywhatkit.playonyt(song) 
        elif 'open google' in query.lower(): 
            speak("here you go!!") 
            webbrowser.open("www.google.com")
        elif 'search' in query.lower(): 
            speak("What do you want me to search for?") 
            search_query = takeCommand().lower() 
            if search_query: 
                url = f"https://www.google.com/search?q={search_query}" 
                webbrowser.get().open(url)
                speak(f"Here are the search results for {search_query}") 
            else:  
                speak("Sorry, I didn't catch that. Could you please repeat?") 
        elif 'the time' in query.lower(): 
            strTime = datetime.datetime.now().strftime("%H:%M:%S") 
            speak(f"the time is {strTime}")
            print(strTime)
        elif 'joke' in query.lower(): 
            s = pyjokes.get_joke() 
            speak(s) 
            print(s) 
        elif 'thanks' in query.lower(): 
            speak("Its my pleasure to help you sir")
        elif 'play music' in query.lower(): 
            speak("here you go!!")
            music_dir = "C:\\Users\AMULA SHARATH KUMAR\\Music" 
            songs = os.listdir(music_dir) 
            print(songs)
            for i in songs: 
                os.startfile(os.path.join(music_dir,i)) 
        elif 'shutdown system' in query.lower(): 
            speak("Hold On a Sec ! Your system is on its way to shut down")
            os.system('shutdown /s /t 0') 
        elif 'search' in query or 'play' in query.lower(): 
            query = query.replace("search", " ") 
            query = query.replace("play", " ") 
            webbrowser.open(query)
        elif 'locate me' in query.lower(): 
            location = query
            speak("user asked to locate")
            url = 'http://ipinfo.io/json'
            response = urlopen(url)
            data = json.load(response)
            speak(f"You are currently in {data['city']}")
            print(f"You are currently in {data['city']}")
        elif "temperature" in query.lower():
            search = "temperature" 
            url = f"https://www.google.com/search?q={search}" 
            r =requests.get(url) 
            data = BeautifulSoup(r.text, "html.parser") 
            temp = data.find("div", class_="BNeawe").text 
            speak(f"current{search}is {temp}") 
            print(temp) 
        elif "open" in query.lower():
            open_app(query.lower())
        elif 'goodbye' in query or 'bye' in query or 'stop that' in query: 
            speak("Thanks for your time and Goodbye!")
            os._exit(0)
        else:
            convo.send_message(query)
            print(convo.last.text)
            speak(convo.last.text)

How's it going?
Listening...
Recognizing..
Unable to Recognize your voice.
Understood.

Listening...
